# CHAPTER10 SQLite 데이터베이스 사용하기

- SQLite
- 데이터베이스 연결하기
- 테이블 생성하기
- 데이터 삽입
- 데이터 선택하기
- 기존 데이터 갱신하기
- 데이터 삭제하기

---

### 10.0 들어가기전에 정리

- 쉽게 말해 파이썬을 이용해 SQL에 접근하는 것입니다.


- 사용되는 원리
    - sqlite3를 호출합니다.
    - connect를 이용해 데이터베이스에 연결합니다.
    - cursor를 이용해 SQL구문을 사용해 데이터베이스를 관리합니다.

### 10.1 SQLite

- SQLite는 디스크 파일 기반의 데이터베이스입니다.
- 별도의 데이터베이스를 준비해야 할 필요가 없어 데이터베이스를 제대로 구축하기 전 프로토타이핑하기에 매우 좋습니다.
- 파일뿐 아니라 메모리에 데이터베이스를 생성해서 사용할 수도 있습니다.

### 10.2 데이터베이스 연결하기

- connect( )로 연결하고 cursor( )로 데이터베이스를 동작시켜서 작업하는 형식입니다.

In [1]:
import sqlite3

# 데이터베이스 파일이 저장될 경로와 파일 이름을 써서 데이터베이스에 연결
con = sqlite3.connect("examples/db.sqlite")

# 유니코드 인코딩 문제가 발생하면 해당 코드의 주석을 해제하고 실행합니다.
con.text_factory = str

# 메모리에서 직접 데이터베이스를 이용합니다.
con = sqlite3.connect(":memory:")

# 데이터베이스를 동작하기 위한 Cursor 객체를 생성합니다.
# 데이터베이스를 사용하기 위한 마지막 준비입니다.
cur = con.cursor()

### 10.3 테이블 생성하기 (CREATE)

- 쿼리문을 사용하기 위해 Cursor클래스의 execute( )를 사용합니다.

In [2]:
cur.execute("""CREATE TABLE hanbit_books(
               title varchar(100),
               author varchar(100),
               translator varchar(100),
               pub_date date,
               isbn varchar(100))""")

# commit을 안해도 상관없지만 명시해 주는것이 혹시모를 에러를 방지할 수 있습니다.
con.commit()

### 10.4 데이터 삽입 (INSERT)

In [3]:
# 테이블에 데이터 넣기
cur.execute("INSERT INTO hanbit_books values (?, ?, ?, ?, ?)",
            # 리스트 내부의 아이템 순서대로 채워집니다.
           ("책 이름", "저자 이름", "번역자 이름", "2016-08-22", "9788968480011"))

In [4]:
# 이름 있는 파라미터 사용하기
query_str = "insert into hanbit_books values (:title, :title, :title, :pub_date, :isbn)"
params = {
    "title" : "책 이름",
    "pub_date" : "2017-10-12",
    "isbn" : 9788968480022
}

cur.execute(query_str, params)

#### 10.4.1 여러개의 데이터 넣기

: 대량의 데이터를 한번에 넣기 위해서 executemany( )를 사용합니다.

In [5]:
import csv

# csv 파일을 열겠습니다.
csv_file = open("examples/book_list.csv")

# csv리더로 읽겠습니다.
csv_reader = csv.reader(csv_file)

# csv파일을 리스트 형식으로 바꿉니다.
book_list = list(csv_reader)

# csv header를 제거합니다.
book_list = book_list[1:]

# 최초 크롤링할 때 있었던 저자 혹은 번역자 데이터의 앞뒤 공백을 제거합니다.
for item in book_list:
    item[1] = item[1].strip()
    item[2] = item[2].strip()
    
# 제대로 되었는지 출력해보겠습니다.
print(book_list[0])
print(book_list[1])
print(book_list[2])
print(book_list[3])
print(book_list[4])

['리얼 블라디보스톡  PLUS 우수리스크 [2019~2020년 최신판]', '강한나', '', '2018-10-22', '9791188007196']
['처음 시작하는 R 데이터 분석', '강전희 , 엄동란', '', '2018-10-05', '9791162241202']
['SWedu, 앱 인벤터', '이종원', '', '2018-06-04', '9791156643968']
['이것이 C#이다', '박상현', '', '2018-07-01', '9791162240861']
['머신러닝 실무 프로젝트', '아리가 미치아키(有賀 康顕) , 나카야마 신타(中山 心太) , 니시바야시 다카시(西林 孝)', '심효섭', '2018-06-05', '9791162240816']


In [6]:
# 데이터베이스에 book_list 넣겠습니다.
cur.executemany("insert into hanbit_books values (?, ?, ?, ?, ?)", book_list)

# 데이터를 넣었으면 확정하기 위해 commit합니다.
con.commit()

### 10.5 데이터 선택하기 (SELECT)

In [7]:
# 쿼리문을 사용해서 데이터 추출하겠습니다.
cur.execute("SELECT * FROM hanbit_books WHERE author = ?", ("강한나",))
print(cur.fetchone())

('리얼 블라디보스톡  PLUS 우수리스크 [2019~2020년 최신판]', '강한나', '', '2018-10-22', '9791188007196')


In [8]:
# 튜플을 이용한 데이터를 선택하겠습니다.
query_str = "SELECT * FROM hanbit_books WHERE author = :name"
params = {"name" : "강한나"}

cur.execute(query_str, params)

In [9]:
# SELECT 쿼리문 실행 결과입니다.
result = list(cur.fetchall())

for row in result:
    print(row)

('리얼 블라디보스톡  PLUS 우수리스크 [2019~2020년 최신판]', '강한나', '', '2018-10-22', '9791188007196')


### 10.6 기존 데이터 갱신하기 (UPDATE)

In [10]:
# author를 찾아서 isbn을 업데이트 하는 쿼리문입니다.
query_str = "UPDATE hanbit_books SET isbn = :isbn WHERE author = :name"
params = {
    # name이 강한나인 데이터의 isbn을 9999999999999로 수정하겠습니다.
    "isbn" : 9999999999999,
    "name" : "강한나"}

cur.execute(query_str, params)

In [11]:
# 갱신되었는지 확인하겠습니다.
cur.execute("SELECT * FROM hanbit_books WHERE author = ?", ("강한나",))
print(cur.fetchone())

('리얼 블라디보스톡  PLUS 우수리스크 [2019~2020년 최신판]', '강한나', '', '2018-10-22', '9999999999999')


### 10.7 데이터 삭제하기 (DELETE)

In [12]:
# DELETE구문을 이용하겠습니다.
cur.execute("DELETE FROM hanbit_books WHERE author = ?", ("강한나",))
print(cur.fetchone())

None


In [13]:
# 테이블을 삭제하겠습니다.
cur.execute("DROP TABLE hanbit_books")

# 데이터베이스 연결을 종료하겠습니다.
con.close()